In [1]:
import os, random
import pandas as pd
import numpy as np

In [2]:
def score_clean(c1):
    os.makedirs('data/course', exist_ok=True)
    file_score = 'data/score/score_list_{}_all.xlsx'.format(str(c1['course_id']))
    file_user  = 'data/course/{}_user_code.tsv'.format(c1['course_id'])
    if not os.path.isfile(file_score):return 0
    score = pd.read_excel(file_score, header=7)
    user  = pd.read_csv(file_user, sep = '\t')
    if 'score' in user.columns.tolist():return 1
    score = score[['帳號', '原始成績']]
    score.rename({'帳號':'user_code', '原始成績':'score'}, axis=1, inplace=True)
    score = score[score['score'].notna()]
    user = user.merge(score, how='inner', on='user_code')
    user.to_csv('data/course/{}_user_code.tsv'.format(c1['course_id']), sep = '\t', index=False)
    return 1

if __name__ == '__main__':
    course = pd.read_excel('data/other/course_list.xlsx', 'course')
    semester = pd.read_excel('data/other/course_list.xlsx', 'semester')
    course['for_train'] = course.apply(score_clean, axis = 1)
    ## 儲存檔案
    with pd.ExcelWriter("data/other/course_list.xlsx") as writer:
        course.to_excel(writer, sheet_name="course", index=False)
        semester.to_excel(writer, sheet_name="semester", index=False)

In [3]:

def getStd(column):
    if column.dtypes == object:
        return pd.Series({'col': column.name, 'std': 0, 'min': 0, })
    column_1 = column[column > 0]
    # print(column.name, len(column_1))
    return pd.Series({
        'col': column.name, 
        'std': column_1.to_numpy().std() if (len(column_1)>0) else 0, 
        'min': column.min(), 
    })

def score_range(score, group_num=20):
    ## label smoothing
    r = int((100 / group_num) // 1)
    s_dict = {}
    sd_1, sd_2 = 0.7, 0.15
    accum = 0
    for s in range(group_num):
        s_dict_val = 0.0
        if (score >= (s*r))&(score < (s*r+r)):
            s_dict_val = sd_1
        elif (score >= (s*r-r))&(score < (s*r)):
            s_dict_val = sd_2
        elif (score >= (s*r+r))&(score < (s*r+r*2)):
            s_dict_val = sd_2
        s_dict[str(s*r)+'~'+str(s*r+r-1)] = s_dict_val
        accum += s_dict_val
    for k in s_dict.keys():
        s_dict[k] /= accum
    return pd.Series(s_dict)


In [4]:
def for_train():
    course = pd.read_excel('data/other/course_list.xlsx', 'course')
    semester = pd.read_excel('data/other/course_list.xlsx', 'semester')
    course = course.merge(semester, on = 'semester')
    course = course[course['for_train'] == 1]
    os.makedirs('data/train', exist_ok=True)
    os.makedirs('model_pytorch', exist_ok=True)
    data_score_x = pd.DataFrame()
    data_score_y = pd.DataFrame()
    data_score_z = pd.DataFrame()
    blacklist = ['B10524008'] ## 數值異常
    for cn in course['course_name'].drop_duplicates():
        cn_data_score_x = pd.DataFrame()
        cn_course = course[course['course_name'] == cn]
        for c in cn_course['course_id']:
            c_course = course[course['course_id'] == c].iloc[0]
            for w in range((c_course['end_at']-c_course['start_at']).days//7):
                w+=1
                cn_data_score_x = pd.concat([cn_data_score_x, pd.read_csv('data/clean/{}w{}_accum.csv'.format(str(c),str(w)))])
        cn_data_score_x = cn_data_score_x[~cn_data_score_x['user_code'].isin(blacklist)]
        cn_data_score_x.fillna(0, inplace=True)
        cn_data_score_x.reset_index(drop=True, inplace=True)
        cn_data_score_x['group'] = random.choices(['training', 'validation', 'testing'], [6, 2, 2], k=len(cn_data_score_x))
        cn_data_score_x.drop(columns=['user_id', 'user_name'], inplace=True) ## 根據保密協定
        ## score predict
        cn_data_score_z = cn_data_score_x['score']
        cn_data_score_y = cn_data_score_z.apply(lambda x: score_range(x, 20))
        cn_data_score_z = pd.DataFrame({'score':cn_data_score_z.tolist()})
        cn_data_score_y = pd.DataFrame(cn_data_score_y)
        cn_data_score_z['group'] = cn_data_score_x['group']
        cn_data_score_y['group'] = cn_data_score_x['group']
        ## 儲存個課程的檔案
        cn_data_score_x.to_csv('data/train/ScorePredict_x_{}.csv'.format(cn), index=False)
        cn_data_score_y.to_csv('data/train/ScorePredict_y_{}.csv'.format(cn), index=False)
        cn_data_score_z.to_csv('data/train/ScorePredict_z_{}.csv'.format(cn), index=False)
        ## 合併檔案
        data_score_x = pd.concat([data_score_x, cn_data_score_x])
        data_score_y = pd.concat([data_score_y, cn_data_score_y])
        data_score_z = pd.concat([data_score_z, cn_data_score_z])
    ## 填補空白處
    data_score_x.fillna(0, inplace=True)
    data_std = data_score_x.apply(getStd, axis=0).T
    ## 去掉不需要的特徵
    data_std = data_std[(data_std['std'] > 0) & (data_std['min'] >= 0)]
    data_std = data_std[~data_std['col'].isin(['score', 'course_code', 'group', 'user_id'])]
    ## 儲存所有資料
    data_std.to_csv('data/train/normalize.csv', index=False)
    data_std.to_csv('model_pytorch/normalize.csv', index=False)
    data_score_x.to_csv('data/train/ScorePredict_x.csv', index=False)
    data_score_y.to_csv('data/train/ScorePredict_y.csv', index=False)
    data_score_z.to_csv('data/train/ScorePredict_z.csv', index=False)

if __name__ == '__main__':
    for_train()

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
